In [1]:
# encoding=utf8
# coding: utf-8

In [3]:
from __future__ import division

import re
import pandas as pd
import numpy as np
import scipy
import math
import json
import pickle
from collections import Counter
from bs4 import BeautifulSoup
import sklearn
import xgboost as xgb
import nltk
#nltk.download('stopwords')

import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english', ignore_stopwords=True)
tokenizer = RegexpTokenizer(r'\s+', gaps=True)

In [2]:
with open(sys.argv[1]) as f:
    text = f.read()

IOError: [Errno 2] No such file or directory: '-f'

In [4]:
### LOCAL
approved_ex_path = r'C:\Users\Artur_Zahreba\Desktop\WorkFusion\P&G\Approval By Email\training_set_txt\Approved\FW  NRO application  Sanya addtional testers tag on PO#G0314-PCA2017057 HTB(SK-II).txt'
other_ex_path = r'C:\Users\Artur_Zahreba\Desktop\WorkFusion\P&G\Approval By Email\training_set_txt\Other\NRO approvals-2.txt'

with open(other_ex_path) as f:
    text = f.read()
    
print(text)

From: WingYan, Choi
Sent: Tue Jul 26 04:57:03 CEST 2016
To: Ong, Cherie; Meng, FanRu; onetravelretail, Ion; Yip, Desmond
Cc: 
Subject: RE: NRO approvals
Hi Cherie, 

 

We don�t have any samples in DC , you can have those leftover in office.

 

Thank you.

 

Clef  

 

From: Ong, Cherie 
Sent: Tuesday, July 26, 2016 10:56 AM
To: Meng, FanRu; onetravelretail, Ion; WingYan, Choi; Yip, Desmond
Subject: NRO approvals

 

Hi Fanru,

 

Please kindly help to advise approval for the attached NROs, thanks!

 

1.       Kris shop party workshop testers

2.       SEA training GS gift 

 

 

Clef and Desmond, please advise stocks supply, thanks!

 

 

 

 

 



 





In [ ]:
model_path = '/home/tomcat/ml/models/'
model_type = 'xgb'
model_name = '2classes_xgb_tfidf_v1.model'

bst = xgb.Booster(model_file=model_path + model_type +'/' + model_name)

In [5]:
### LOCAL
model_path = r'C:\Users\Artur_Zahreba\Desktop\text-classification-model\Scripts\models\xgb\2classes_xgb_tfidf_v1.model'
bst = xgb.Booster(model_file=model_path)

In [ ]:
vectorizer_path = '/home/tomcat/ml/vectorizers/'
vectorizer_type = 'tfidf'
vectorizer_name = 'vectorizer2.pk'

with open(vectorizer_path + vectorizer_type + '/' + vectorizer_name, 'rb') as fin:
    vectorizer = pickle.load(fin)

In [6]:
### LOCAL
with open(r'C:\Users\Artur_Zahreba\Desktop\text-classification-model\Scripts\vectorizers\tfidf\vectorizer2.pk', 'rb') as vec_file:
    vectorizer = pickle.load(vec_file)

In [7]:
lable_dict = {'Approved': 1, 'Other': 0}

inv_lable_dict = dict(zip(lable_dict.values(), lable_dict.keys()))
inv_lable_dict

{0: 'Other', 1: 'Approved'}

In [8]:
text = text.decode('utf8', 'ignore')

texts = [text]

texts = map(lambda x: stemmer.stem(x), texts)
texts = map(lambda x: tokenizer.tokenize(x), texts)
texts = map(lambda x: re.sub(r'\s\d+\s', ' ', ' '.join(x)), texts)


X = np.array(texts)

X = vectorizer.transform(X)
X.shape

(1, 216)

In [9]:
# X_lens = np.ones([X.shape[0], 1])
# X_dense = np.array(map(lambda x, y: 1.0 * len(x) / y, text, X_lens)).reshape(X.shape[0], 1)

# X = scipy.sparse.hstack([X, X_lens, X_dense])
X.shape

(1, 216)

In [15]:
DX = xgb.DMatrix(X)
# result = bst.predict(DX, output_margin=True)
result = bst.predict(DX)
result

array([ 0.], dtype=float32)

In [11]:
inv_lable_dict[result[0]]

TypeError: unhashable type: 'numpy.ndarray'

In [12]:
def softmax(array):
    e = map (lambda x: sum(map (lambda y: math.pow(math.e, y), x)), array)
    array = map (lambda x, y: map (lambda z: math.pow(math.e, z)/y, x), array, e)
    return np.array(array)

In [13]:
softmax(result)

array([[ 0.50155758,  0.49844242]])

In [16]:
result = inv_lable_dict[result[0]]

true_out = map(json.dumps, result)
true_out

['"O"', '"t"', '"h"', '"e"', '"r"']

In [ ]:
true_out = map(json.dumps, true_out)

In [ ]:
f = open(sys.argv[2], 'w')

for i in range(batch_size):
    sys.stdout.write(true_out[i])
    f.write(true_out[i])

f.close()